In [37]:
from sentence_transformers import SentenceTransformer, util
import os
import csv
import pickle
import time
import faiss
import numpy as np


In [1]:
import json 

file=open('data/train.json','r')

with file:
    data=json.load(file)

In [2]:
len(data)


76560

In [3]:
data[0].keys()

dict_keys(['document', 'messages', 'answers'])

In [6]:
len(data[2]['document'])

7920

In [7]:
import numpy as np

embeddings_small=np.load('data/small_embeddings.npy')
queries_small=np.load('data/small_queries.npy')

In [30]:
embeddings_large=np.load('data/large_embeddings.npy')
queries_large=np.load('data/large_queries.npy')

In [31]:
embeddings_large.shape

(76560, 512)

In [32]:
queries_large.shape

(76560, 512)

In [18]:
import time
import sklearn as sk
from sklearn.neighbors import KDTree

t1=time.time()
tree_small = KDTree(embeddings_small, leaf_size=4000)
t2=time.time()
print('KD Tree build time : {0:3f}[s]'.format(t2-t1))

KD Tree build time : 0.415210[s]


In [33]:
t1=time.time()
tree_large = KDTree(embeddings_large, leaf_size=4000)
t2=time.time()
print('KD Tree build time : {0:3f}[s]'.format(t2-t1))

KD Tree build time : 0.754699[s]


In [34]:
queries_large[:100].shape

(100, 512)

In [21]:
t1=time.time()
ind = tree_small.query(queries_small[:1], k=1,return_distance=False,dualtree=True)
t2=time.time()
print('KD Tree query time 1 docs : {0:3f}[s]'.format(t2-t1))

KD Tree query time 1 docs : 0.034355[s]


In [35]:
t1=time.time()
ind = tree_large.query(queries_large[:1], k=1,return_distance=False,dualtree=True)
t2=time.time()
print('KD Tree query time 1 docs : {0:3f}[s]'.format(t2-t1))

KD Tree query time 1 docs : 0.062520[s]


In [22]:
t1=time.time()
ind = tree_small.query(queries_small[:100], k=1,return_distance=False,dualtree=True)
t2=time.time()
print('KD Tree query time 100 docs : {0:3f}[s]'.format(t2-t1))

KD Tree query time 100 docs : 2.335918[s]


In [24]:
t1=time.time()
ind = tree_small.query(queries_small[:1000], k=1,return_distance=False,dualtree=True)
t2=time.time()
print('KD Tree query time 1000 docs : {0:3f}[s]'.format(t2-t1))

KD Tree query time 1000 docs : 22.956999[s]


In [36]:
t1=time.time()
ind = tree_large.query(queries_large[:1000], k=1,return_distance=False,dualtree=True)
t2=time.time()
print('KD Tree query time 1000 docs : {0:3f}[s]'.format(t2-t1))

KD Tree query time 1000 docs : 34.776457[s]


In [37]:
import ollama
from ollama import chat

In [38]:
data[0]['messages'][0]['content']

'How many Americans are part of the federal food assistance program? '

In [39]:
data[0]['answers']

['31 million']

In [40]:
messages=list()
answers=list()
questions=list()
for ux,ix in enumerate(ind):
  question=data[ux]['messages'][0]['content']
  text=data[ix[0]]['document']
  messages.append({
    'role': 'user',
    'content': f'Respond the following question : "{question}" based on the following text : "{text}"',
  })
  questions.append(question)
  answers.append(data[ux]['answers'])

In [41]:
num=0
for q,a,m in zip(questions,answers,messages):
    if num==10:
        break
    response = chat('phi3', messages=[m])
    print(f'Question : {q}')
    print(f'Real Answers : {a}')
    print(f'Response : {response['message']['content']}')
    num+=1
    print('----------------')

Question : How many Americans are part of the federal food assistance program? 
Real Answers : ['31 million']
Response : Based on the provided text, approximately 46 million Americans are part of the federal food assistance program known as the Supplemental Nutrition Assistance Program (SNAP).
----------------
Question : How much did Sean Callebs live on? 
Real Answers : ['$176']
Response : The text does not provide any information on how much Sean Callebs lives on. Therefore, we cannot answer the question based on this text.
----------------
Question : What percentage of people in the USA are part of the federal food assistance program? 
Real Answers : ['more than one in 10']
Response : The text provided does not specify an exact percentage of people in the USA who are part of the federal food assistance program, also known as the Supplemental Nutrition Assistance Program (SNAP). However, it states that SNAP provides about 46 million Americans with debit cards to buy food each month. 